In [ ]:
import numpy
import matplotlib.pyplot as plt
import matplotlib.dates as mpldates
%matplotlib inline
import scipy.interpolate as sci
import scipy.optimize as sco
import json
from astropy.time import Time as astrotime
import datetime

# Load data

Load IC86 data from epinat, which should be the usual IC86-I (2011) PS sample, but pull corrected and OneWeights corrected by number of events generated.

In [ ]:
exp = np.load("data/IC86_I_data.npy")
mc = np.load("data/IC86_I_mc.npy")

# Get data livetime

Generate from good run list as stated here:
- http://icecube.wisc.edu/~coenders/html/build/html/ic86-bdt/muonL3.html
- https://wiki.icecube.wisc.edu/index.php/IC86_I_Point_Source_Analysis/Data_and_Simulation

It should be 332.61 days as stated by jefeintzeig and scoenders.
We create one bin per included run, with exactly that width.
Excluded runs are those with too high/low rate and without everything marked "good".

Livetime ist a bit higher, because we used a newer runlist from iclive instead of the old non-json v1.4.
See M&F folder for a script that parses that list.

In [ ]:
jsonFile = open('data/ic86-i-goodrunlist.json', 'r')
grlist = json.load(jsonFile)
jsonFile.close()

In [ ]:
# This is a list of dicts (one dict per run)
runs = grlist["runs"]
# This is a dict of arrays (all run values in an array per keyword)
run_dict = dict(zip(runs[0].keys(), zip(*[r.values() for r in runs])))
for k in run_dict.keys():
    run_dict[k] = np.array(run_dict[k])

In [ ]:
# Now compile runs as stated on jfenitzeigs page

# Transform livetimes to MJD floats
start_mjd = astrotime(run_dict["good_tstart"]).mjd
stop_mjd = astrotime(run_dict["good_tstop"]).mjd

# Create recarry to apply mask, only keep start, stop and runID
dtype = [("start_mjd", np.float), ("stop_mjd", np.float), ("runID", np.int)]
run_arr = np.array(list(zip(start_mjd, stop_mjd, run_dict["run"])), dtype=dtype)

# Note: The last 2 runs aren't included anyway, so he left them out in
# the reported run list. This fits here, as the other 4 runs are found
# in the list.
exclude_rate = [120028, 120029, 120030, 120087, 120156, 120157]
i3good = run_dict["good_i3"] == True
itgood = run_dict["good_it"] == True
ratebad = np.in1d(run_dict["run"], exclude_rate)

# Include if it & i3 good and rate is good
include = i3good & itgood & ~ratebad
inc_run_arr = run_arr[include]

# Get the total and per run livetimes in mjd
runtimes_mjd = inc_run_arr["stop_mjd"] - inc_run_arr["start_mjd"]
livetime = np.sum(runtimes_mjd)

print("IC86-I livetime from v1.5 (iclive): ", livetime)

# Use the official livetime for the rest
livetime = 332.61

# Bin BG according to runlist

Each run is one bin in the bg rate vs time plot.
The rate is normed to Hertz by dividing through the bin sizes in seconds.

In [ ]:
# Store events in bins with run borders
exp_times = exp["timeMJD"]
start_mjd = inc_run_arr["start_mjd"]
stop_mjd = inc_run_arr["stop_mjd"]

tot = 0
evts_in_run = {}
for start, stop , runid in zip(start_mjd, stop_mjd, inc_run_arr["runID"]):
    mask = (exp_times >= start) & ( exp_times < stop)
    evts_in_run[runid] = exp[mask]
    tot += np.sum(mask)
    
# Crosscheck, if we got all events and counted nothing double
print("Do we have all events? ", tot == len(exp))

In [ ]:
# Create binmids and histogram values in each bin
binmids = 0.5 * (start_mjd + stop_mjd)
h = np.zeros(len(binmids), dtype=np.float)

for i, evts in enumerate(evts_in_run.values()):
    h[i] = len(evts)
    
# Create plot arrays
xerr = runtimes_mjd / 2.
yerr = np.sqrt(h)

# Show in Hertz, so go from MJD days to seconds in bin widths
secsinday = 24. * 60. * 60
norm = (stop_mjd - start_mjd) * secsinday
h_norm = h / norm
# Poisson errors just get scaled
yerr_norm = yerr / norm

# Fit a poly to the rate
weights = np.ones_like(yerr)
weights[yerr_norm == 0] = 0
weights[yerr_norm != 0] = 1 / yerr[yerr_norm != 0]
def f(x, a, b, c):
    return np.sin(a * (x - b)) + c
normed = (binmids - binmids.min()) / (binmids.max() - binmids.min())
res = sco.curve_fit(f=f, xdata=normed, ydata=h_norm, p0=[0.001, 0., 0.005], sigma=1/weights)

In [ ]:
plt.plot(normed, f(normed, *res[0]))

In [ ]:
# Plot like mrichman did on p. 113
fig, ax = plt.subplots(1, 1)

# Show dates on x axis
datetimes = astrotime(binmids, format="mjd").to_datetime()
dates = mpldates.date2num([dt.date() for dt in datetimes])

# Every month, first day
months = mpldates.MonthLocator(bymonth=np.arange(1, 13), bymonthday=1)
monthsFmt = mpldates.DateFormatter("%b %Y")
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)

ax.errorbar(dates, h_norm, fmt=".", xerr=xerr, yerr=yerr_norm)
ax.set_xlabel("Date")
ax.set_ylabel("Rate in HZ")
ax.set_xlim(dates[0], dates[-1])

# Plot polyfit
delta_days = (datetimes[-1] - datetimes[0]).days
xdatetimes = [datetimes[0] + datetime.timedelta(days=int(x))for x in
              np.arange(0, delta_days)]
xtimes_mjd = astrotime(xdatetimes).mjd
normed = (xtimes_mjd - binmids.min()) / (binmids.max() - binmids.min())
y = f(normed, *res[0])
xdates = mpldates.date2num([xd.date() for xd in xdatetimes])
ax.plot(xdates, y, "r-", zorder=5)
ax.axhline(np.average(h_norm, weights=weights), 0, 1, color="k", ls="--", zorder=5)

# Autoprettify main xlabels
fig.autofmt_xdate(rotation=60)

# Show mjd on top
def ax2ticker(x):
    dates = mpldates.num2date(x)
    mjd = astrotime(dates).mjd
    return mjd
ax2 = ax.twiny()
ax2.set_xticks(ax.get_xticks())
ax2.set_xbound(ax.get_xbound())
ax2.set_xticklabels(ax2ticker(ax.get_xticks()),
                    rotation=60, horizontalalignment="left")
ax2.set_xlabel("MJD")